<a href="https://colab.research.google.com/github/ceb263/nhl/blob/main/xG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports and input data
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import log_loss, roc_auc_score

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

pd.set_option('display.max_rows', 150)

In [2]:
!unzip data_2012-2019.zip

Archive:  data_2012-2019.zip
  inflating: pbp_2019.pkl            
  inflating: __MACOSX/._pbp_2019.pkl  
  inflating: pbp_2012.pkl            
  inflating: __MACOSX/._pbp_2012.pkl  
  inflating: pbp_2013.pkl            
  inflating: __MACOSX/._pbp_2013.pkl  
  inflating: pbp_2014.pkl            
  inflating: __MACOSX/._pbp_2014.pkl  
  inflating: pbp_2015.pkl            
  inflating: __MACOSX/._pbp_2015.pkl  
  inflating: pbp_2016.pkl            
  inflating: __MACOSX/._pbp_2016.pkl  
  inflating: pbp_2017.pkl            
  inflating: __MACOSX/._pbp_2017.pkl  
  inflating: pbp_2018.pkl            
  inflating: __MACOSX/._pbp_2018.pkl  


In [2]:
# read data
plays = pd.read_pickle('pbp_2012.pkl')
plays['Season'] = 2012
plays2013 = pd.read_pickle('pbp_2013.pkl')
plays2013['Season'] = 2013
plays2014 = pd.read_pickle('pbp_2014.pkl')
plays2014['Season'] = 2014
plays2015 = pd.read_pickle('pbp_2015.pkl')
plays2015['Season'] = 2015
plays2016 = pd.read_pickle('pbp_2016.pkl')
plays2016['Season'] = 2016
plays2017 = pd.read_pickle('pbp_2017.pkl')
plays2017['Season'] = 2017
plays2018 = pd.read_pickle('pbp_2018.pkl')
plays2018['Season'] = 2018
plays2019 = pd.read_pickle('pbp_2019.pkl')
plays2019['Season'] = 2019
plays = pd.concat([plays, plays2013], ignore_index=True)
plays = pd.concat([plays, plays2014], ignore_index=True)
plays = pd.concat([plays, plays2015], ignore_index=True)
plays = pd.concat([plays, plays2016], ignore_index=True)
plays = pd.concat([plays, plays2017], ignore_index=True)
plays = pd.concat([plays, plays2018], ignore_index=True)
plays = pd.concat([plays, plays2019], ignore_index=True)

del plays2013, plays2014, plays2015, plays2016, plays2017, plays2018, plays2019

In [3]:
# data preprocessing
def preprocess_plays(df):
    # remove null location data
    df = df.loc[~df['xC'].isnull()]
    df = df.loc[~df['yC'].isnull()]

    # get previous event time and location
    df = df.sort_values(by=['Game_Id','Period','Seconds_Elapsed'])
    df['prev_Game_Id'] = df['Game_Id'].shift(1)
    df['prev_Period'] = df['Period'].shift(1)
    df['keepPrev'] = ((df['prev_Game_Id']==df['Game_Id']) & (df['prev_Period']==df['Period'])).astype(int)
    df['prev_Event'] = df['Event'].shift(1)
    df['prev_Seconds_Elapsed'] = df['Seconds_Elapsed'].shift(1)
    df['prev_xC'] = df['xC'].shift(1)
    df['prev_yC'] = df['yC'].shift(1)
    df.at[df['keepPrev']==0, ['prev_Event','prev_Seconds_Elapsed','prev_xC','prev_yC']] = np.NaN
    df = df.loc[~df['prev_Event'].isnull()]

    # get time elapsed, and distance from previous event
    df['timeSincePrev'] = df['Seconds_Elapsed'] - df['prev_Seconds_Elapsed']
    df['distanceSincePrev'] = np.sqrt(np.square(df['xC']-df['prev_xC']) + np.square(df['yC']-df['prev_yC']))
    df['yDistanceSincePrev'] = np.abs(df['yC'] - df['prev_yC'])

    # remove invalid data
    df = df.loc[(df['timeSincePrev']>0) | (df['timeSincePrev'].isnull())]

    # filter for only shots and shot attempts
    df = df.loc[df['Event'].isin(['BLOCK','MISS','SHOT','GOAL'])]

    # filter for only most common game states
    df = df.loc[df['Strength'].isin(['5x5','4x5','3x5','5x4','4x4','5x3','4x3','6x5','5x6','3x4','3x3'])]

    # get previous shot time and location, and then calculate derived metrics
    df['prevShot_Game_Id'] = df['Game_Id'].shift(1)
    df['prevShot_Period'] = df['Period'].shift(1)
    df['keepPrevShot'] = ((df['prevShot_Game_Id']==df['Game_Id']) & (df['prevShot_Period']==df['Period'])).astype(int)
    df['prevShot_Seconds_Elapsed'] = df['Seconds_Elapsed'].shift(1)
    df['prevShot_xC'] = df['xC'].shift(1)
    df['prevShot_yC'] = df['yC'].shift(1)
    df['prevShot_Ev_Team'] = df['Ev_Team'].shift(1)
    df['prevShot_sameTeam'] = (df['prevShot_Ev_Team']==df['Ev_Team']).astype(int)
    df.at[df['keepPrevShot']==0, ['prevShot_Seconds_Elapsed','prevShot_xC','prevShot_yC','prevShot_Ev_Team']] = np.NaN
    df['timeSincePrevShot'] = df['Seconds_Elapsed'] - df['prevShot_Seconds_Elapsed']
    df['distanceSincePrevShot'] = np.sqrt(np.square(df['xC']-df['prevShot_xC']) + np.square(df['yC']-df['prevShot_yC']))
    df['yDistanceSincePrevShot'] = np.abs(df['yC'] - df['prevShot_yC'])

    # adjust shot locations so everything is on the same side of the ice
    # TODO this isn't quite right - shots from the D zone (into an empty net, for example), will not be adjusted correctly
    df['loc_adjust_factor'] = (((df['xC']>0).astype(int).astype(float)) - 0.5) * 2
    df['x_adj'] = df['xC']*df['loc_adjust_factor']
    df['y_adj'] = df['yC']*df['loc_adjust_factor']
    df['prev_loc_adjust_factor'] = (((df['prev_xC']>0).astype(int).astype(float)) - 0.5) * 2
    df['prev_x_adj'] = df['prev_xC']*df['prev_loc_adjust_factor']
    df['prev_y_adj'] = df['prev_yC']*df['prev_loc_adjust_factor']
    df['prevShot_loc_adjust_factor'] = (((df['prevShot_xC']>0).astype(int).astype(float)) - 0.5) * 2
    df['prevShot_x_adj'] = df['prevShot_xC']*df['prevShot_loc_adjust_factor']
    df['prevShot_y_adj'] = df['prevShot_yC']*df['prevShot_loc_adjust_factor']

    # fill nulls
    df[['prevShot_Seconds_Elapsed','prevShot_yC','prevShot_y_adj','prevShot_xC']] = \
        df[['prevShot_Seconds_Elapsed','prevShot_yC','prevShot_y_adj','prevShot_xC']].fillna(0)
    df[['prevShot_x_adj','distanceSincePrevShot','yDistanceSincePrevShot']] = df[['prevShot_x_adj','distanceSincePrevShot','yDistanceSincePrevShot']].fillna(-1)
    df[['timeSincePrevShot']] = df[['timeSincePrevShot']].fillna(1200)

    # fix time since prev shot if prev shot was in another period
    df.at[df['timeSincePrevShot']<0, 'timeSincePrevShot'] = 1200

    # adjust score to be score for and against, instead of home and away
    df['homeTeamShot'] = (df['Home_Team']==df['Ev_Team']).astype(int)
    df['scoreFor'] = (df['Home_Score']*df['homeTeamShot']) + (df['Away_Score']*(1-df['homeTeamShot']))
    df['scoreAgainst'] = (df['Away_Score']*df['homeTeamShot']) + (df['Home_Score']*(1-df['homeTeamShot']))
    df['scoreDiff'] = df['scoreFor'] - df['scoreAgainst']

    # add target variable
    df['goal'] = (df['Event']=='GOAL').astype(int)

    return df

In [4]:
shots = preprocess_plays(plays)

In [5]:
# select feature and target columns
X = shots[['Period','Seconds_Elapsed','scoreFor','scoreAgainst','scoreDiff','xC','yC','prev_Seconds_Elapsed','prev_xC','prev_yC','timeSincePrev','distanceSincePrev',
           'yDistanceSincePrev','prevShot_Seconds_Elapsed','prevShot_xC','prevShot_yC','prevShot_sameTeam','timeSincePrevShot','distanceSincePrevShot',
           'yDistanceSincePrevShot','x_adj','y_adj','prev_x_adj','prev_y_adj','prevShot_x_adj','prevShot_y_adj','Strength','Ev_Zone','Type','prev_Event','goal','Season']]
y = shots[['goal','Season']]

# train/test split
#[X_train, X_test, y_train, y_test] = train_test_split(X, y, test_size=0.3, random_state=26)
X_train = X.loc[X['Season']!=2019]
X_test = X.loc[X['Season']==2019]
y_train = y.loc[y['Season']!=2019]['goal'].values
y_test = y.loc[y['Season']==2019]['goal'].values

# categorical feature encodings
mean_codes_strength = X_train.groupby(['Strength'])['goal'].mean().to_dict()
mean_codes_zone = X_train.groupby(['Ev_Zone'])['goal'].mean().to_dict()
mean_codes_type = X_train.groupby(['Type'])['goal'].mean().to_dict()
mean_codes_prevEvent = X_train.groupby(['prev_Event'])['goal'].mean().to_dict()

tot_count = X_train['goal'].count()
count_codes_strength = (X_train.groupby(['Strength'])['goal'].count()/tot_count).to_dict()
count_codes_zone = (X_train.groupby(['Ev_Zone'])['goal'].count()/tot_count).to_dict()
count_codes_type = (X_train.groupby(['Type'])['goal'].count()/tot_count).to_dict()
count_codes_prevEvent = (X_train.groupby(['prev_Event'])['goal'].count()/tot_count).to_dict()

X_train['Strength_meanEnc'] = X_train['Strength'].map(mean_codes_strength)
X_test['Strength_meanEnc'] = X_test['Strength'].map(mean_codes_strength)
X_train['Ev_Zone_meanEnc'] = X_train['Ev_Zone'].map(mean_codes_zone)
X_test['Ev_Zone_meanEnc'] = X_test['Ev_Zone'].map(mean_codes_zone)
X_train['Type_meanEnc'] = X_train['Type'].map(mean_codes_type)
X_test['Type_meanEnc'] = X_test['Type'].map(mean_codes_type)
X_train['prev_Event_meanEnc'] = X_train['prev_Event'].map(mean_codes_prevEvent)
X_test['prev_Event_meanEnc'] = X_test['prev_Event'].map(mean_codes_prevEvent)
X_train['Strength_countEnc'] = X_train['Strength'].map(count_codes_strength)
X_test['Strength_countEnc'] = X_test['Strength'].map(count_codes_strength)
X_train['Ev_Zone_countEnc'] = X_train['Ev_Zone'].map(count_codes_zone)
X_test['Ev_Zone_countEnc'] = X_test['Ev_Zone'].map(count_codes_zone)
X_train['Type_countEnc'] = X_train['Type'].map(count_codes_type)
X_test['Type_countEnc'] = X_test['Type'].map(count_codes_type)
X_train['prev_Event_countEnc'] = X_train['prev_Event'].map(count_codes_prevEvent)
X_test['prev_Event_countEnc'] = X_test['prev_Event'].map(count_codes_prevEvent)

X_train = X_train.drop(['goal','Strength','Ev_Zone','Type','prev_Event','Season'],1).values
X_test = X_test.drop(['goal','Strength','Ev_Zone','Type','prev_Event','Season'],1).values

'''
strengthOneHot = LabelBinarizer()
zoneOneHot = LabelBinarizer()
typeOneHot = LabelBinarizer()
prevEventOneHot = LabelBinarizer()
X = np.concatenate((X, strengthOneHot.fit_transform(shots['Strength']), zoneOneHot.fit_transform(shots['Ev_Zone']), typeOneHot.fit_transform(shots['Type']), 
                   prevEventOneHot.fit_transform(shots['prev_Event'])), axis=1)
'''

# apply scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

# Model Testing

In [6]:
[_, X_subsample, _, y_subsample] = train_test_split(X_train, y_train, test_size=0.01, random_state=26)

In [7]:
X_subsample.shape

(8292, 34)

In [11]:
model_test = KNeighborsClassifier()

param_dist = {
    'n_neighbors' : [300,500]
}

random_search = GridSearchCV(model_test, param_dist, scoring=['neg_log_loss','roc_auc'], refit='neg_log_loss', cv=3, return_train_score=True)
random_search.fit(X_subsample, y_subsample)

report_cols = ['mean_test_neg_log_loss','mean_test_roc_auc']+['param_'+param for param in param_dist]
report = pd.DataFrame(random_search.cv_results_)[report_cols].sort_values(by='mean_test_neg_log_loss', ascending=False)
report

,mean_test_neg_log_loss,mean_test_roc_auc,param_n_neighbors
0,-0.169279,0.740524,300
1,-0.171191,0.734494,500


# Selected Model Training

In [12]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)
preds_gb = model_gb.predict_proba(X_test)[:,1]
print ('LogLoss score: {}'.format(str(log_loss(y_test, preds_gb))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, preds_gb))))

LogLoss score: 0.1656247228929607
AUC score: 0.8176907076768725


In [26]:
pickle.dump(model_gb, open('gb.pkl', 'wb'))

In [13]:
model_gb2 = GradientBoostingClassifier(max_depth=3, min_weight_fraction_leaf=0.001)
model_gb2.fit(X_train, y_train)
preds_gb2 = model_gb2.predict_proba(X_test)[:,1]
print ('LogLoss score: {}'.format(str(log_loss(y_test, preds_gb2))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, preds_gb2))))

LogLoss score: 0.1658067883554617
AUC score: 0.8163280936824684


In [25]:
pickle.dump(model_gb2, open('gb2.pkl', 'wb'))

In [14]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_lr = model_lr.predict_proba(X_test)[:,1]
print ('LogLoss score: {}'.format(str(log_loss(y_test, preds_lr))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, preds_lr))))

LogLoss score: 0.17561467421598137
AUC score: 0.7684700372074639


In [24]:
pickle.dump(model_lr, open('lr.pkl', 'wb'))

In [15]:
model_rf = RandomForestClassifier(min_weight_fraction_leaf=0.0001, max_depth=20)
model_rf.fit(X_train, y_train)
preds_rf = model_rf.predict_proba(X_test)[:,1]
print ('LogLoss score: {}'.format(str(log_loss(y_test, preds_rf))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, preds_rf))))

LogLoss score: 0.1646318095332512
AUC score: 0.819229789699326


In [23]:
pickle.dump(model_rf, open('rf.pkl', 'wb'))

In [16]:
model_knn = KNeighborsClassifier(n_neighbors=300)
model_knn.fit(X_train, y_train)
preds_knn = model_knn.predict_proba(X_test)[:,1]
print ('LogLoss score: {}'.format(str(log_loss(y_test, preds_knn))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, preds_knn))))

LogLoss score: 0.17299654552684413
AUC score: 0.7850207966543266


In [22]:
pickle.dump(model_knn, open('knn.pkl', 'wb'))

In [17]:
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential

In [18]:
def create_model_nn(input_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(52))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy')
    return model

In [19]:
model_nn = create_model_nn(X_train.shape[1])
model_nn.fit(X_train, y_train, verbose=1, epochs=10, batch_size=1024)
preds_nn = model_nn.predict(X_test)[:,0]
print ('LogLoss score: {}'.format(str(log_loss(y_test, np.clip(preds_nn, a_min=10e-5, a_max = 1-10e-5)))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, np.clip(preds_nn, a_min=10e-5, a_max = 1-10e-5)))))

Epoch 1/10
810/810 [==============================] - 6s 3ms/step - loss: 0.1841
Epoch 2/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1668
Epoch 3/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1642
Epoch 4/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1640
Epoch 5/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1628
Epoch 6/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1619
Epoch 7/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1605
Epoch 8/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1612
Epoch 9/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1611
Epoch 10/10
810/810 [==============================] - 2s 3ms/step - loss: 0.1602
LogLoss score: 0.168170889242463
AUC score: 0.8151714673784262


In [29]:
model_nn.save('nn.h5')

In [20]:
def create_model_nn2(input_dim):
    model = Sequential()
    model.add(Dense(1024, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy')
    return model

In [21]:
model_nn2 = create_model_nn2(X_train.shape[1])
model_nn2.fit(X_train, y_train, verbose=1, epochs=10, batch_size=1024)
preds_nn2 = model_nn2.predict(X_test)[:,0]
print ('LogLoss score: {}'.format(str(log_loss(y_test, np.clip(preds_nn2, a_min=10e-5, a_max = 1-10e-5)))))
print ('AUC score: {}'.format(str(roc_auc_score(y_test, np.clip(preds_nn2, a_min=10e-5, a_max = 1-10e-5)))))

Epoch 1/10
810/810 [==============================] - 4s 4ms/step - loss: 0.1753
Epoch 2/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1625
Epoch 3/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1610
Epoch 4/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1602
Epoch 5/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1601
Epoch 6/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1585
Epoch 7/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1586
Epoch 8/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1597
Epoch 9/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1591
Epoch 10/10
810/810 [==============================] - 3s 4ms/step - loss: 0.1574
LogLoss score: 0.16457861175969563
AUC score: 0.8173687588194842


In [30]:
model_nn2.save('nn2.h5')